# NOTEBOOK TFM MASTER IA

Predicción de cambio de empleo de ingenieros de Data Science

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.preprocessing import StandardScaler
from feature_engine.imputation import CategoricalImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.model_selection import GridSearchCV
import torch
import pickle
import random
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
import keras

In [2]:
#Se carga el dataset

url = "C:\\Users\\guill\\Desktop\\Master IA y Data Science\\TFM\\dataset"
archivo = pd.read_csv(url + "\ds_job.csv")

In [ ]:
#Se exploran las variables y valores

archivo.head()

In [ ]:
archivo.describe()

In [ ]:
archivo.info()

In [ ]:
for cols in archivo.columns:
    print("Feature: " + cols)
    a = pd.Categorical(archivo[cols])
    print(a)

In [271]:
#Se divide en datasets de entrenamiento y validacion

X_train, X_test, y_train, y_test = train_test_split(archivo.drop(["empleado_id","target"] , axis = 1), archivo["target"], test_size=0.2, random_state=42)


In [272]:
#Se divide en features categóricas y numéricas

features_num = ["indice_desarrollo_ciudad","horas_formacion"]
X_train_int = X_train[features_num]
X_train_cat = X_train.drop(features_num , axis = 1)
X_train_cat["tamano_compania"] = pd.DataFrame(X_train_cat["tamano_compania"].apply(lambda x : '0-10' if x == '<10' else x))
X_train_cat["ultimo_nuevo_trabajo"] = pd.DataFrame(X_train_cat["ultimo_nuevo_trabajo"].apply(lambda x : '4+' if x == '>4' else x))
features_cat = X_train_cat.columns.values.tolist()

In [273]:
#Rellenar valores NAN con mediana (numéricos) y moda (categóricos). También aplicamos un StandarScaler para las variables
#numéricas

imputer = SimpleImputer(strategy="median") 
X_train_int = pd.DataFrame(imputer.fit_transform(X_train_int),columns = features_num)
imputer = CategoricalImputer(imputation_method='frequent')
X_train_cat = imputer.fit_transform(X_train_cat)

In [274]:
#Se modifican valores que contengan <> para evitar errores durante el onehotencoder

bins = pd.DataFrame(X_train_cat["experiencia"].apply(lambda x : '25' if x == '>20' else x))
bins = pd.DataFrame(bins["experiencia"].apply(lambda x : '0' if x == '<1' else x))
bins["experiencia"] = pd.to_numeric(bins["experiencia"])
labels = ['0-2', '2-5', '5-10', '10-20', '20+']
X_train_cat["experiencia"] = pd.cut(bins.experiencia, bins = 5, labels = labels)

In [275]:
le = LabelEncoder()
ohe = OneHotEncoder()

#No se emplearán ni la variable ciudad ni la variable género.
#La variable ciudad son 123 valores categóricos distintos, por lo que añadiríamos muchas columnas al dataset, cuando 
#ya tenemos una variable que nos indica el indice de desarrollo de la ciudad que puede ser un indicativo más importante
#que la ciudad en si
#En la variable género teníamos 4500 filas sin valor, y hay 10 veces más hombres que mujeres, por lo que o eliminamos esas 
#filas sin valor o no deberíamos tener en cuenta esta feature. De esta forma también evitamos consideraciones o sesgo 
#por genero que puede ser éticamente incorrecto

#cat_encoded = le.fit_transform(X_train_cat["genero"].astype(str))
#cat_ohe = ohe.fit_transform(cat_encoded.reshape(-1,1))
#X_train_df_genero = pd.DataFrame(cat_ohe.toarray(),columns = "genero_" + le.classes_)

#cat_encoded = le.fit_transform(X_train_cat["ciudad"].astype(str))
#cat_ohe = ohe.fit_transform(cat_encoded.reshape(-1,1))
#X_train_df_ciudad = pd.DataFrame(cat_ohe.toarray(),columns = "ciudad_" + le.classes_)

cat_encoded = le.fit_transform(X_train_cat["experiencia_relevante"].astype(str))
cat_ohe = ohe.fit_transform(cat_encoded.reshape(-1,1))
X_train_df_experiencia_relevante = pd.DataFrame(cat_ohe.toarray(),columns = "experiencia_relevante_" + le.classes_)

cat_encoded = le.fit_transform(X_train_cat["universidad_matriculado"].astype(str))
cat_ohe = ohe.fit_transform(cat_encoded.reshape(-1,1))
X_train_df_universidad = pd.DataFrame(cat_ohe.toarray(),columns = "universidad_matriculado_" + le.classes_)

cat_encoded = le.fit_transform(X_train_cat["nivel_educacion"].astype(str))
cat_ohe = ohe.fit_transform(cat_encoded.reshape(-1,1))
X_train_df_nivel_educacion = pd.DataFrame(cat_ohe.toarray(),columns = "nivel_educacion_" + le.classes_)

cat_encoded = le.fit_transform(X_train_cat["educacion"].astype(str))
cat_ohe = ohe.fit_transform(cat_encoded.reshape(-1,1))
X_train_df_educacion = pd.DataFrame(cat_ohe.toarray(),columns = "educacion_" + le.classes_)

cat_encoded = le.fit_transform(X_train_cat["experiencia"].astype(str))
cat_ohe = ohe.fit_transform(cat_encoded.reshape(-1,1))
X_train_df_experiencia = pd.DataFrame(cat_ohe.toarray(),columns = "experiencia_" + le.classes_)

cat_encoded = le.fit_transform(X_train_cat["tamano_compania"].astype(str))
cat_ohe = ohe.fit_transform(cat_encoded.reshape(-1,1))
X_train_df_tamano_compania = pd.DataFrame(cat_ohe.toarray(),columns = "tamano_compania_" + le.classes_)

cat_encoded = le.fit_transform(X_train_cat["tipo_compania"].astype(str))
cat_ohe = ohe.fit_transform(cat_encoded.reshape(-1,1))
X_train_df_tipo_compania = pd.DataFrame(cat_ohe.toarray(),columns = "tipo_compania_" + le.classes_)

cat_encoded = le.fit_transform(X_train_cat["ultimo_nuevo_trabajo"].astype(str))
cat_ohe = ohe.fit_transform(cat_encoded.reshape(-1,1))
X_train_df_ultimo_nuevo_trabajo = pd.DataFrame(cat_ohe.toarray(),columns = "ultimo_nuevo_trabajo_" + le.classes_)

#Juntamos todo en el DataFrame de entrenamiento final

X_train_df = pd.concat([X_train_int,X_train_df_experiencia_relevante, X_train_df_universidad, X_train_df_nivel_educacion, X_train_df_educacion, X_train_df_experiencia, X_train_df_tamano_compania, X_train_df_tipo_compania, X_train_df_ultimo_nuevo_trabajo],axis=1)

In [ ]:
#Se visualiza la matriz de correlaciones por si se observara alguna relación o dependencia entre variables que permitiera 
#reducir el número de features a tener en cuenta

correlacion = X_train_df.corr()
correlacion.style.background_gradient(cmap="RdBu_r", low = 0, high = 0)

In [14]:
#El primer modelo a probar sera un SVC, se emplea GridSearchCV para tunning de los hiperparámetros

svc_model = SVC()
param_grid = {'C': [0.1, 1, 10, 100, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']} 
svc_clf = GridSearchCV(svc_model, param_grid, scoring='roc_auc', refit = True, verbose = 2)
svc_result = svc_clf.fit(X_train_df, y_train)
print("Best: %f using %s" % (svc_result.best_score_, svc_result.best_params_))  

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   8.8s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=  11.0s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=  12.6s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=  10.1s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=  12.3s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=   6.5s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=   6.4s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=   6.3s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=   6.6s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=   6.4s
[CV] END ......................C=0.1, gamma=0.01, kernel=rbf; total time=   6.4s
[CV] END ......................C=0.1, gamma=0.0

[CV] END ........................C=1000, gamma=1, kernel=rbf; total time=  51.0s
[CV] END ........................C=1000, gamma=1, kernel=rbf; total time= 1.0min
[CV] END ........................C=1000, gamma=1, kernel=rbf; total time=  54.8s
[CV] END ........................C=1000, gamma=1, kernel=rbf; total time=  50.0s
[CV] END ......................C=1000, gamma=0.1, kernel=rbf; total time= 1.1min
[CV] END ......................C=1000, gamma=0.1, kernel=rbf; total time= 1.2min
[CV] END ......................C=1000, gamma=0.1, kernel=rbf; total time= 1.3min
[CV] END ......................C=1000, gamma=0.1, kernel=rbf; total time= 1.2min
[CV] END ......................C=1000, gamma=0.1, kernel=rbf; total time= 1.2min
[CV] END .....................C=1000, gamma=0.01, kernel=rbf; total time=  27.2s
[CV] END .....................C=1000, gamma=0.01, kernel=rbf; total time=  26.4s
[CV] END .....................C=1000, gamma=0.01, kernel=rbf; total time=  28.1s
[CV] END ...................

In [250]:
#Se crea modelo con los mejores parámetros y se comprueba la ROC, la matriz de confusión

svc_final = SVC(C = 1000, gamma = 0.0001, kernel = 'rbf')
svc_trained = svc_final.fit(X_train_df,y_train)
pred_svc = svc_trained.predict(X_train_df)
print(classification_report(y_train, pred_svc))
print(confusion_matrix(y_train,pred_svc))
print("ROC: " + str(roc_auc_score(y_train, pred_svc)))

#El modelo predice para todo un 0, por lo que no es adecuado

              precision    recall  f1-score   support

         0.0       0.75      1.00      0.86     11501
         1.0       0.00      0.00      0.00      3825

    accuracy                           0.75     15326
   macro avg       0.38      0.50      0.43     15326
weighted avg       0.56      0.75      0.64     15326

[[11501     0]
 [ 3825     0]]
ROC: 0.5


C:\Users\guill\anaconda3\envs\torch_env\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\guill\anaconda3\envs\torch_env\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\guill\anaconda3\envs\torch_env\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [26]:
#A continuación se prueba un XGBoost Classifier, de nuevo se realiza un tunning de los hiperparámetros, pero en este 
#caso iremos cambiando en lotes de parámetros

param_test1 = {
 'max_depth':range(3,10,1),
 'min_child_weight':range(1,11,1)
}

xgb_clf = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='roc_auc',n_jobs=4, cv=5, verbose=3, refit=True)

xgb_result = xgb_clf.fit(X_train_df, y_train)
print("Best: %f using %s" % (xgb_result.best_score_, xgb_result.best_params_))  

Fitting 5 folds for each of 70 candidates, totalling 350 fits


C:\Users\guill\anaconda3\envs\torch_env\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[08:19:12] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Best: 0.785263 using {'max_depth': 4, 'min_child_weight': 9}


In [34]:
param_test2 = {
    'gamma':[i/100.0 for i in range(0,100)]
}
xgb_clf2 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=4,
 min_child_weight=9, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test2, scoring='roc_auc',n_jobs=4, cv=5, verbose=3, refit=True)

xgb_result2 = xgb_clf2.fit(X_train_df, y_train)
print("Best: %f using %s" % (xgb_result2.best_score_, xgb_result2.best_params_))  

Fitting 5 folds for each of 100 candidates, totalling 500 fits


C:\Users\guill\anaconda3\envs\torch_env\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[19:16:55] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Best: 0.785517 using {'gamma': 0.35}


In [36]:
param_test3 = {
    'reg_alpha':[0, 0.001, 0.005, 0.01, 0.05]
}
xgb_clf3 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=4,
 min_child_weight=9, gamma=0.35, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test3, scoring='roc_auc',n_jobs=4, cv=5, verbose=3, refit=True)

xgb_result3 = xgb_clf3.fit(X_train_df, y_train)
print("Best: %f using %s" % (xgb_result3.best_score_, xgb_result3.best_params_))  

Fitting 5 folds for each of 5 candidates, totalling 25 fits


C:\Users\guill\anaconda3\envs\torch_env\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[19:23:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Best: 0.785523 using {'reg_alpha': 0.001}


In [37]:
param_test4 = {
    'subsample':[i/10.0 for i in range(6,20)],
    'colsample_bytree':[i/10.0 for i in range(6,20)]
}
xgb_clf4 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=4,
 min_child_weight=9, gamma=0.35, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,reg_alpha = 0.001, seed=27), 
 param_grid = param_test4, scoring='roc_auc',n_jobs=4, cv=5, verbose=3, refit=True)

xgb_result4 = xgb_clf4.fit(X_train_df, y_train)
print("Best: %f using %s" % (xgb_result4.best_score_, xgb_result4.best_params_))

Fitting 5 folds for each of 196 candidates, totalling 980 fits


C:\Users\guill\anaconda3\envs\torch_env\lib\site-packages\sklearn\model_selection\_search.py:922: UserWarning: One or more of the test scores are non-finite: [0.78499778 0.78608408 0.78524451 0.78508831 0.78546556        nan
        nan        nan        nan        nan        nan        nan
        nan        nan 0.78490154 0.78569248 0.78530548 0.78521801
 0.78499225        nan        nan        nan        nan        nan
        nan        nan        nan        nan 0.78360136 0.7849021
 0.78552327 0.78429665 0.78594852        nan        nan        nan
        nan        nan        nan        nan        nan        nan
 0.78324484 0.78395234 0.78452065 0.78409275 0.78452125        nan
        nan        nan        nan        nan        nan        nan
        nan        nan 0.78303119 0.78363665 0.78355374 0.78354733
 0.78412279        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan      

[19:26:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Best: 0.786084 using {'colsample_bytree': 0.6, 'subsample': 0.7}


In [205]:
param_test5 = {
    'n_estimators':list(range(100,5000,100))
}
xgb_clf5 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=4,
 min_child_weight=9, gamma=0.35, subsample=0.7, colsample_bytree=0.6,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,reg_alpha = 0.001, seed=27), 
 param_grid = param_test5, scoring='roc_auc',n_jobs=4, cv=5, verbose=3, refit=True)

xgb_result5 = xgb_clf5.fit(X_train_df, y_train)
print("Best: %f using %s" % (xgb_clf5.best_score_, xgb_clf5.best_params_))

Fitting 5 folds for each of 49 candidates, totalling 245 fits


C:\Users\guill\anaconda3\envs\torch_env\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[11:09:35] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


NameError: name 'xgb_result_final' is not defined

In [240]:
#Creamos el modelo definitivo con el tunning de cada lote de parámetros, y se comprueban ROC y matriz de confusión

xgb_clf = XGBClassifier( learning_rate =0.1, n_estimators=100, max_depth=4,
 min_child_weight=9, gamma=0.35, subsample=0.7, colsample_bytree=0.6,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,reg_alpha = 0.001, seed=27)
xgb_clf_trained = xgb_clf.fit(X_train_df, y_train)
pred_xgb = xgb_clf_trained.predict(X_train_df)
print(classification_report(y_train, pred_xgb))
print(confusion_matrix(y_train,pred_xgb))
print("ROC: " + str(roc_auc_score(y_train, pred_xgb)))

C:\Users\guill\anaconda3\envs\torch_env\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:13:16] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
              precision    recall  f1-score   support

         0.0       0.83      0.91      0.87     11501
         1.0       0.62      0.43      0.51      3825

    accuracy                           0.79     15326
   macro avg       0.72      0.67      0.69     15326
weighted avg       0.78      0.79      0.78     15326

[[10496  1005]
 [ 2192  1633]]
ROC: 0.6697721994052237


In [6]:
#El siguiente modelo será un KNeighbor. De nuevo se emplea GridSearchCV

kn_param = {'n_neighbors': list(range(1,20)),
            'leaf_size': [20,40,100,250,400],
            'algorithm':['auto'],
            'weights': ["uniform", "distance"],
            'metric': ('minkowski', 'chebyshev')}

kn_search = GridSearchCV(KNeighborsClassifier(), kn_param, cv = 5, scoring = 'roc_auc', n_jobs = -1, verbose = 3)
kn_result = kn_search.fit(X=X_train_df, y=y_train)
print("Best KNeighbour: %f using %s" % (kn_result.best_score_, kn_result.best_params_)) 

Fitting 5 folds for each of 380 candidates, totalling 1900 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:   39.9s
[Parallel(n_jobs=-1)]: Done 104 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 264 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done 488 tasks      | elapsed:  8.1min
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed: 12.6min
[Parallel(n_jobs=-1)]: Done 1128 tasks      | elapsed: 18.2min
[Parallel(n_jobs=-1)]: Done 1544 tasks      | elapsed: 24.3min
[Parallel(n_jobs=-1)]: Done 1900 out of 1900 | elapsed: 30.1min finished


Best KNeighbour: 0.749528 using {'algorithm': 'auto', 'leaf_size': 100, 'metric': 'minkowski', 'n_neighbors': 19, 'weights': 'uniform'}


In [239]:
#Se crea modelo con los mejores hiperparámetros 

kn_clf_final = KNeighborsClassifier( algorithm = 'auto', leaf_size= 100, metric = 'minkowski', n_neighbors = 19, weights= 'uniform')
kn_clf_trained = kn_clf_final.fit(X=X_train_df, y=y_train)
pred_kn = kn_clf_trained.predict(X_train_df)
print(classification_report(y_train, pred_kn))
print(confusion_matrix(y_train,pred_kn))
print("ROC: " + str(roc_auc_score(y_train, pred_kn)))

              precision    recall  f1-score   support

         0.0       0.83      0.92      0.87     11501
         1.0       0.63      0.42      0.51      3825

    accuracy                           0.79     15326
   macro avg       0.73      0.67      0.69     15326
weighted avg       0.78      0.79      0.78     15326

[[10551   950]
 [ 2203  1622]]
ROC: 0.6707253873348893


In [14]:
#El último modelo de ML será un Random Forest

n_estimators = [int(x) for x in np.linspace(start = 20, stop = 100, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 50, num = 10)]
max_depth.append(None)
min_samples_split = [2, 5]
min_samples_leaf = [1, 2]
bootstrap = [True, False]
rf_param = {'n_estimators': n_estimators,
            'max_features': max_features,
            'max_depth': max_depth,
            'min_samples_split': min_samples_split,
            'min_samples_leaf': min_samples_leaf,
            'bootstrap': bootstrap}

forest_clf_search = GridSearchCV(RandomForestClassifier(), rf_param, cv = 5, scoring = 'roc_auc', n_jobs = -1, verbose = 3)
forest_result=forest_clf_search.fit(X=X_train_df, y=y_train)
print("Best Forest Classifier: %f using %s" % (forest_result.best_score_, forest_result.best_params_)) 

Fitting 5 folds for each of 1760 candidates, totalling 8800 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:   26.0s
[Parallel(n_jobs=-1)]: Done 104 tasks      | elapsed:   47.5s
[Parallel(n_jobs=-1)]: Done 264 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 488 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 1128 tasks      | elapsed:  5.6min
[Parallel(n_jobs=-1)]: Done 1544 tasks      | elapsed:  7.8min
[Parallel(n_jobs=-1)]: Done 2024 tasks      | elapsed: 10.4min
[Parallel(n_jobs=-1)]: Done 2568 tasks      | elapsed: 13.8min
[Parallel(n_jobs=-1)]: Done 3176 tasks      | elapsed: 17.1min
[Parallel(n_jobs=-1)]: Done 3848 tasks      | elapsed: 20.7min
[Parallel(n_jobs=-1)]: Done 4584 tasks      | elapsed: 24.4min
[Parallel(n_jobs=-1)]: Done 5384 tasks      | elapsed: 28.0min
[Parallel(n_jobs=-1)]: Done 6248 tasks      | elapsed: 33.6min
[Parallel(n_jobs=-1)]: Done 7176 tasks      | 

Best KNeighbour: 0.782215 using {'bootstrap': True, 'max_depth': 14, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 82}


In [276]:
forest_clf_final = RandomForestClassifier( bootstrap = True, max_depth = 14, max_features = 'sqrt', min_samples_leaf = 2, min_samples_split = 2, n_estimators = 82)
forest_clf_trained = forest_clf_final.fit(X=X_train_df, y=y_train)
pred_forest = forest_clf_trained.predict(X_train_df)
classification_report(y_train, pred_forest)
print(classification_report(y_train, pred_forest))
print(confusion_matrix(y_train,pred_forest))
print("ROC: " + str(roc_auc_score(y_train, pred_forest)))

              precision    recall  f1-score   support

         0.0       0.84      0.95      0.89     11501
         1.0       0.74      0.47      0.58      3825

    accuracy                           0.83     15326
   macro avg       0.79      0.71      0.73     15326
weighted avg       0.82      0.83      0.81     15326

[[10880   621]
 [ 2021  1804]]
ROC: 0.7088193410859982


In [277]:
#Se guardan todos los modelos para poder ponerlos en produccion posteriormente

filename = 'SVC.pkl'
pickle.dump(svc_trained, open(filename, 'wb'))
filename = 'XGBoost.pkl'
pickle.dump(xgb_clf_trained, open(filename, 'wb'))
filename = 'KNeighbor.pkl'
pickle.dump(kn_clf_trained, open(filename, 'wb'))
filename = 'RandomForest.pkl'
pickle.dump(forest_clf_trained, open(filename, 'wb'))

In [226]:
#Se declaran clases y funciones para entrenamiento y evaluacion de redes neuronales con Pytorch

class Flatten(torch.nn.Module):

    def forward(self, x):

        return x.view(x.shape[0], -1)

def pred_f(x, forw):

    with torch.no_grad():

        _, y_pred = torch.max(forw(x), dim=1)

    return y_pred

def score_f(x, y, forw):

    with torch.no_grad():

        y_pred = pred_f(x, forw)

        score = torch.sum(y_pred == y).item() / len(y)

    return score

def loss_f(x, y, forw):

    return torch.nn.functional.mse_loss(forw(x), y)

def init_f(forw):

    for nam, param in forw.named_parameters():

        if 'weight' in nam:

            torch.nn.init.xavier_normal_(param) 

        else:

            param.data.fill_(0.01)

def t_step(x, y, forw, opt):

    opt.zero_grad() # reseteamos los valores acumulados de las derivadas
    

    loss = loss_f(x, y, forw) # calculamos la 'loss' con los resultados 

    out = loss.item() # almacenamos nuestro valor de loss para devolverlo como resultado intermedio

    loss.backward() # propagamos las derivadas de la 'loss' con respecto a nuestros parámetros

    opt.step() # ejecutamos un paso de nuestro optimizador

    return out

def do_seeds(sn):
    np.random.seed(sn)
    torch.manual_seed(sn)
    random.seed(sn)
do_seeds(0)

In [242]:
#Se generan tensores de los datos de entrenamiento

tX_train = torch.tensor(np.array(X_train_df), requires_grad=False, dtype=torch.float)
ty_train = torch.tensor(np.array(y_train), requires_grad=False, dtype=torch.float)  

In [243]:
#Se crea red neuronal con 7 capas

nn1 = torch.nn.Sequential(
    torch.nn.Linear(43,64),
    torch.nn.ReLU(),
    torch.nn.Linear(64,128),
    torch.nn.ReLU(),
    torch.nn.Linear(128,512),
    torch.nn.ReLU(),
    torch.nn.Linear(512, 4096),
    torch.nn.ReLU(),
    torch.nn.Linear(4096, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 64),  
    torch.nn.ReLU(),
    torch.nn.Linear(64, 1) 
)
do_seeds(29)
init_f(nn1)
print(score_f(tX_train, ty_train, nn1))

0.7504241158815086


In [246]:
#Se entrena red neuronal con optimizador SGD y función de perdida MSE con 50 epochs

opt1 = torch.optim.SGD(nn1.parameters(), lr=0.0001)
mini_b = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(tX_train, ty_train.view(-1,1)), 
                                       batch_size=100, shuffle=True) 
for i in range(50): 
    for x_j, y_j in mini_b:
        out = t_step(x_j, y_j, nn1, opt1) 
    if i%1 == 0:
        print("Epoch: {}, Loss: {:.4f}".format(i, out))

Epoch: 0, Loss: 0.1035
Epoch: 1, Loss: 0.1326
Epoch: 2, Loss: 0.1760
Epoch: 3, Loss: 0.1787
Epoch: 4, Loss: 0.1271
Epoch: 5, Loss: 0.2395
Epoch: 6, Loss: 0.2623
Epoch: 7, Loss: 0.1457
Epoch: 8, Loss: 0.1578
Epoch: 9, Loss: 0.2107
Epoch: 10, Loss: 0.1231
Epoch: 11, Loss: 0.1623
Epoch: 12, Loss: 0.1477
Epoch: 13, Loss: 0.1866
Epoch: 14, Loss: 0.1963
Epoch: 15, Loss: 0.1710
Epoch: 16, Loss: 0.1842
Epoch: 17, Loss: 0.1421
Epoch: 18, Loss: 0.2051
Epoch: 19, Loss: 0.1607
Epoch: 20, Loss: 0.1561
Epoch: 21, Loss: 0.1278
Epoch: 22, Loss: 0.1709
Epoch: 23, Loss: 0.1851
Epoch: 24, Loss: 0.0709
Epoch: 25, Loss: 0.1916
Epoch: 26, Loss: 0.1053
Epoch: 27, Loss: 0.1403
Epoch: 28, Loss: 0.1993
Epoch: 29, Loss: 0.1762
Epoch: 30, Loss: 0.1976
Epoch: 31, Loss: 0.1461
Epoch: 32, Loss: 0.1132
Epoch: 33, Loss: 0.1658
Epoch: 34, Loss: 0.1870
Epoch: 35, Loss: 0.1622
Epoch: 36, Loss: 0.2062
Epoch: 37, Loss: 0.1431
Epoch: 38, Loss: 0.1090
Epoch: 39, Loss: 0.1790
Epoch: 40, Loss: 0.1239
Epoch: 41, Loss: 0.1674
Ep

In [245]:
print(score_f(tX_train, ty_train, nn1))

0.7504241158815086


In [ ]:
predicciones = pred_f(tX_train,nn1)
predicciones

#Se realiza prediccion a partir de los datos de entrenamiento, y se comprueba que siempre predice un 0, por lo que no es una
#red válida

In [43]:
#Se prueba a entreanr una red más sencilla, con sólo una capa de entrada y otra de salida, y función de activación ReLu

nn2 = torch.nn.Sequential(
    torch.nn.Linear(43,64),
    torch.nn.ReLU(),
    torch.nn.Linear(64, 1) 
)
do_seeds(29)
init_f(nn2)
print(score_f(tX_train, ty_train, nn1))

0.7504241158815086


In [44]:
#Se entrena red neuronal con optimizador Adam en esta ocasion y función de perdida MSE con 50 epochs

opt2 = torch.optim.Adam(nn2.parameters(), weight_decay=0.008, lr=0.0002)
mini_b = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(tX_train, ty_train.view(-1,1)), 
                                       batch_size=100, shuffle=True) 
for i in range(50): 
    for x_j, y_j in mini_b:
        out = t_step(x_j, y_j, nn2, opt2) 
    if i%1 == 0:
        print("Epoch: {}, Loss: {:.4f}".format(i, out))

Epoch: 0, Loss: 0.1340
Epoch: 1, Loss: 0.1519
Epoch: 2, Loss: 0.0834
Epoch: 3, Loss: 0.1092
Epoch: 4, Loss: 0.1475
Epoch: 5, Loss: 0.1630
Epoch: 6, Loss: 0.2041
Epoch: 7, Loss: 0.1767
Epoch: 8, Loss: 0.1603
Epoch: 9, Loss: 0.1755
Epoch: 10, Loss: 0.1601
Epoch: 11, Loss: 0.1332
Epoch: 12, Loss: 0.1822
Epoch: 13, Loss: 0.0865
Epoch: 14, Loss: 0.1019
Epoch: 15, Loss: 0.1254
Epoch: 16, Loss: 0.1566
Epoch: 17, Loss: 0.1931
Epoch: 18, Loss: 0.1714
Epoch: 19, Loss: 0.1822
Epoch: 20, Loss: 0.1039
Epoch: 21, Loss: 0.1565
Epoch: 22, Loss: 0.1883
Epoch: 23, Loss: 0.1170
Epoch: 24, Loss: 0.1350
Epoch: 25, Loss: 0.1624
Epoch: 26, Loss: 0.2352
Epoch: 27, Loss: 0.1185
Epoch: 28, Loss: 0.1272
Epoch: 29, Loss: 0.1772
Epoch: 30, Loss: 0.1324
Epoch: 31, Loss: 0.1124
Epoch: 32, Loss: 0.1716
Epoch: 33, Loss: 0.1528
Epoch: 34, Loss: 0.1499
Epoch: 35, Loss: 0.1429
Epoch: 36, Loss: 0.1384
Epoch: 37, Loss: 0.2190
Epoch: 38, Loss: 0.1912
Epoch: 39, Loss: 0.1856
Epoch: 40, Loss: 0.2325
Epoch: 41, Loss: 0.2047
Ep

In [45]:
print(score_f(tX_train, ty_train, nn2))

0.7504241158815086


In [41]:
predicciones = pred_f(tX_train,nn2)
predicciones

#De nuevo la red neuronal predice todo 0

tensor([0, 0, 0,  ..., 0, 0, 0])

In [302]:
#Finalmente, se prueba una nueva red neuronal (esta vez creada con Keras) con varias capas lineales y de normalizacion, 
#y se entrena con 2000 epochs

opt = keras.optimizers.Adam(learning_rate=0.001)
modelo = keras.Sequential([
    keras.layers.Dense(300, input_dim=43, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(rate = 0.2),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(rate = 0.2),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(rate = 0.2),
    keras.layers.Dense(1, activation='sigmoid')
])
modelo.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
modelo.fit(X_train_df, y_train, epochs=2000)

Epoch 1/2000
479/479 [==============================] - 2s 2ms/step - loss: 0.6944 - accuracy: 0.6300
Epoch 2/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.5503 - accuracy: 0.7453
Epoch 3/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.5209 - accuracy: 0.7531
Epoch 4/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.5172 - accuracy: 0.7567
Epoch 5/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.4923 - accuracy: 0.7729
Epoch 6/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.4948 - accuracy: 0.7656
Epoch 7/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.4970 - accuracy: 0.7604
Epoch 8/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.4872 - accuracy: 0.7687
Epoch 9/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.4904 - accuracy: 0.7677
Epoch 10/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.491

479/479 [==============================] - 1s 2ms/step - loss: 0.4252 - accuracy: 0.7953
Epoch 159/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.4209 - accuracy: 0.7940
Epoch 160/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.4259 - accuracy: 0.7983
Epoch 161/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.4254 - accuracy: 0.7927
Epoch 162/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.4234 - accuracy: 0.7972
Epoch 163/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.4313 - accuracy: 0.7916
Epoch 164/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.4166 - accuracy: 0.8008
Epoch 165/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.4295 - accuracy: 0.7917
Epoch 166/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.4282 - accuracy: 0.7925
Epoch 167/2000
479/479 [==============================] - 1s 2ms/step - loss: 0

479/479 [==============================] - 1s 2ms/step - loss: 0.4030 - accuracy: 0.8077
Epoch 315/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.4058 - accuracy: 0.8010
Epoch 316/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.4150 - accuracy: 0.7996
Epoch 317/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.4112 - accuracy: 0.8055
Epoch 318/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.3970 - accuracy: 0.8068
Epoch 319/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.4096 - accuracy: 0.8044
Epoch 320/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.3988 - accuracy: 0.8075
Epoch 321/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.4151 - accuracy: 0.7940
Epoch 322/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.4145 - accuracy: 0.8009
Epoch 323/2000
479/479 [==============================] - 1s 2ms/step - loss: 0

479/479 [==============================] - 1s 2ms/step - loss: 0.3937 - accuracy: 0.8085
Epoch 471/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.3943 - accuracy: 0.8127
Epoch 472/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.3936 - accuracy: 0.8103
Epoch 473/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.4047 - accuracy: 0.8066
Epoch 474/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.3951 - accuracy: 0.8099
Epoch 475/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.3919 - accuracy: 0.8105
Epoch 476/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.4017 - accuracy: 0.8071
Epoch 477/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.3910 - accuracy: 0.8101
Epoch 478/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.3942 - accuracy: 0.8129
Epoch 479/2000
479/479 [==============================] - 1s 2ms/step - loss: 0

479/479 [==============================] - 1s 2ms/step - loss: 0.3857 - accuracy: 0.8129
Epoch 627/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.3882 - accuracy: 0.8212
Epoch 628/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.3858 - accuracy: 0.8179
Epoch 629/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.3839 - accuracy: 0.8141
Epoch 630/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.3767 - accuracy: 0.8194
Epoch 631/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.3903 - accuracy: 0.8110
Epoch 632/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.3930 - accuracy: 0.8065
Epoch 633/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.3886 - accuracy: 0.8149
Epoch 634/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.3920 - accuracy: 0.8116
Epoch 635/2000
479/479 [==============================] - 1s 2ms/step - loss: 0

...

Epoch 1406/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.3682 - accuracy: 0.8255
Epoch 1407/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.3698 - accuracy: 0.8227
Epoch 1408/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.3672 - accuracy: 0.8206
Epoch 1409/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.3661 - accuracy: 0.8288
Epoch 1410/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.3608 - accuracy: 0.8255
Epoch 1411/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.3664 - accuracy: 0.8242
Epoch 1412/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.3712 - accuracy: 0.8211
Epoch 1413/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.3762 - accuracy: 0.8230
Epoch 1414/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.3620 - accuracy: 0.8293
Epoch 1415/2000
479/479 [==============================

479/479 [==============================] - 1s 2ms/step - loss: 0.3694 - accuracy: 0.8252
Epoch 1485/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.3791 - accuracy: 0.8171
Epoch 1486/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.3606 - accuracy: 0.8277
Epoch 1487/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.3692 - accuracy: 0.8227
Epoch 1488/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.3631 - accuracy: 0.8271
Epoch 1489/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.3713 - accuracy: 0.8230
Epoch 1490/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.3758 - accuracy: 0.8154
Epoch 1491/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.3651 - accuracy: 0.8251
Epoch 1492/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.3797 - accuracy: 0.8173
Epoch 1493/2000
479/479 [==============================] - 1s 2ms/step 

479/479 [==============================] - 1s 2ms/step - loss: 0.3661 - accuracy: 0.8305
Epoch 1641/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.3633 - accuracy: 0.8260
Epoch 1642/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.3543 - accuracy: 0.8295
Epoch 1643/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.3632 - accuracy: 0.8289
Epoch 1644/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.3651 - accuracy: 0.8260
Epoch 1645/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.3698 - accuracy: 0.8235
Epoch 1646/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.3603 - accuracy: 0.8262
Epoch 1647/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.3631 - accuracy: 0.8260
Epoch 1648/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.3626 - accuracy: 0.8252
Epoch 1649/2000
479/479 [==============================] - 1s 2ms/step 

479/479 [==============================] - 1s 3ms/step - loss: 0.3752 - accuracy: 0.8148
Epoch 1797/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.3488 - accuracy: 0.8365
Epoch 1798/2000
479/479 [==============================] - 1s 3ms/step - loss: 0.3615 - accuracy: 0.8263
Epoch 1799/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.3608 - accuracy: 0.8220
Epoch 1800/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.3633 - accuracy: 0.8300
Epoch 1801/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.3627 - accuracy: 0.8265
Epoch 1802/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.3692 - accuracy: 0.8233
Epoch 1803/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.3622 - accuracy: 0.8271
Epoch 1804/2000
479/479 [==============================] - 1s 2ms/step - loss: 0.3736 - accuracy: 0.8236
Epoch 1805/2000
479/479 [==============================] - 1s 2ms/step 

479/479 [==============================] - 1s 3ms/step - loss: 0.3652 - accuracy: 0.8264
Epoch 1953/2000
479/479 [==============================] - 1s 3ms/step - loss: 0.3614 - accuracy: 0.8250
Epoch 1954/2000
479/479 [==============================] - 2s 3ms/step - loss: 0.3699 - accuracy: 0.8202
Epoch 1955/2000
479/479 [==============================] - 2s 3ms/step - loss: 0.3605 - accuracy: 0.8269
Epoch 1956/2000
479/479 [==============================] - 1s 3ms/step - loss: 0.3730 - accuracy: 0.8231
Epoch 1957/2000
479/479 [==============================] - 1s 3ms/step - loss: 0.3472 - accuracy: 0.8362
Epoch 1958/2000
479/479 [==============================] - 2s 3ms/step - loss: 0.3668 - accuracy: 0.8241
Epoch 1959/2000
479/479 [==============================] - 1s 3ms/step - loss: 0.3566 - accuracy: 0.8327
Epoch 1960/2000
479/479 [==============================] - 1s 3ms/step - loss: 0.3669 - accuracy: 0.8248
Epoch 1961/2000
479/479 [==============================] - 1s 3ms/step 

In [303]:
#Se comprueba resultados de esta nueva red y se comprueba que esta ya no predice que todo es 0. El problema de esta red
#estaría en si se intenta entrenar, el tiempo necesario seria mayor al de otro modelo de ML tradicional

predictions = modelo.predict(X_train_df)
for i in range(len(predictions)):
    if predictions[i][0] > 0.5:
        predictions[i][0] = 1
    else:
        predictions[i] = 0
print(classification_report(y_train, predictions))
print(confusion_matrix(y_train, predictions))

              precision    recall  f1-score   support

         0.0       0.89      0.93      0.91     11501
         1.0       0.75      0.64      0.69      3825

    accuracy                           0.86     15326
   macro avg       0.82      0.79      0.80     15326
weighted avg       0.85      0.86      0.85     15326

[[10693   808]
 [ 1367  2458]]


In [304]:
#Se guarda modelo de la red neuronal
filename = 'NN_Keras'
modelo.save(filename)

INFO:tensorflow:Assets written to: NN_Keras\assets


In [278]:
#Se define funcion para la transformacion de los datos

def pipeline_transformacion(X):
    
    features_num = ["indice_desarrollo_ciudad","horas_formacion"]
    X_int = X[features_num]
    X_cat = X.drop(features_num , axis = 1)
    X_cat["tamano_compania"] = pd.DataFrame(X_cat["tamano_compania"].apply(lambda x : '0-10' if x == '<10' else x))
    X_cat["ultimo_nuevo_trabajo"] = pd.DataFrame(X_cat["ultimo_nuevo_trabajo"].apply(lambda x : '4+' if x == '>4' else x))
    features_cat = X_cat.columns.values.tolist()
    imputer = SimpleImputer(strategy="median") 
    X_int = pd.DataFrame(imputer.fit_transform(X_int),columns = features_num)
    imputer = CategoricalImputer(imputation_method='frequent')
    X_cat = imputer.fit_transform(X_cat)
    bins = pd.DataFrame(X_cat["experiencia"].apply(lambda x : '25' if x == '>20' else x))
    bins = pd.DataFrame(bins["experiencia"].apply(lambda x : '0' if x == '<1' else x))
    bins["experiencia"] = pd.to_numeric(bins["experiencia"])
    labels = ['0-2', '2-5', '5-10', '10-20', '20+']
    X_cat["experiencia"] = pd.cut(bins.experiencia, bins = 5, labels = labels)
    
    le = LabelEncoder()
    ohe = OneHotEncoder()

    cat_encoded = le.fit_transform(X_cat["experiencia_relevante"].astype(str))
    cat_ohe = ohe.fit_transform(cat_encoded.reshape(-1,1))
    X_df_experiencia_relevante = pd.DataFrame(cat_ohe.toarray(),columns = "experiencia_relevante_" + le.classes_)

    cat_encoded = le.fit_transform(X_cat["universidad_matriculado"].astype(str))
    cat_ohe = ohe.fit_transform(cat_encoded.reshape(-1,1))
    X_df_universidad = pd.DataFrame(cat_ohe.toarray(),columns = "universidad_matriculado_" + le.classes_)

    cat_encoded = le.fit_transform(X_cat["nivel_educacion"].astype(str))
    cat_ohe = ohe.fit_transform(cat_encoded.reshape(-1,1))
    X_df_nivel_educacion = pd.DataFrame(cat_ohe.toarray(),columns = "nivel_educacion_" + le.classes_)

    cat_encoded = le.fit_transform(X_cat["educacion"].astype(str))
    cat_ohe = ohe.fit_transform(cat_encoded.reshape(-1,1))
    X_df_educacion = pd.DataFrame(cat_ohe.toarray(),columns = "educacion_" + le.classes_)

    cat_encoded = le.fit_transform(X_cat["experiencia"].astype(str))
    cat_ohe = ohe.fit_transform(cat_encoded.reshape(-1,1))
    X_df_experiencia = pd.DataFrame(cat_ohe.toarray(),columns = "experiencia_" + le.classes_)

    cat_encoded = le.fit_transform(X_cat["tamano_compania"].astype(str))
    cat_ohe = ohe.fit_transform(cat_encoded.reshape(-1,1))
    X_df_tamano_compania = pd.DataFrame(cat_ohe.toarray(),columns = "tamano_compania_" + le.classes_)

    cat_encoded = le.fit_transform(X_cat["tipo_compania"].astype(str))
    cat_ohe = ohe.fit_transform(cat_encoded.reshape(-1,1))
    X_df_tipo_compania = pd.DataFrame(cat_ohe.toarray(),columns = "tipo_compania_" + le.classes_)

    cat_encoded = le.fit_transform(X_cat["ultimo_nuevo_trabajo"].astype(str))
    cat_ohe = ohe.fit_transform(cat_encoded.reshape(-1,1))
    X_df_ultimo_nuevo_trabajo = pd.DataFrame(cat_ohe.toarray(),columns = "ultimo_nuevo_trabajo_" + le.classes_)

    X_df = pd.concat([X_int,X_df_experiencia_relevante, X_df_universidad, X_df_nivel_educacion, X_df_educacion, X_df_experiencia, X_df_tamano_compania, X_df_tipo_compania, X_df_ultimo_nuevo_trabajo],axis=1)
    return X_df

In [305]:
#Se emplea funcion de procesamiento de los datos de validacion

X_test_df = pipeline_transformacion(X_test)

In [262]:
#Se cargan los modelos guardados y se comprueban resultados con datos de validacion

xgb_clf = pickle.load(open('XGBoost.pkl', 'rb'))
prediccion = xgb_clf.predict(X_test_df)
print(classification_report(y_test, prediccion))
print(confusion_matrix(y_test, prediccion))

              precision    recall  f1-score   support

         0.0       0.77      0.97      0.86      2880
         1.0       0.59      0.12      0.20       952

    accuracy                           0.76      3832
   macro avg       0.68      0.55      0.53      3832
weighted avg       0.73      0.76      0.70      3832

[[2803   77]
 [ 839  113]]


In [263]:
kn_clf = pickle.load(open('KNeighbor.pkl', 'rb'))
prediccion = kn_clf.predict(X_test_df)
print(classification_report(y_test, prediccion))
print(confusion_matrix(y_test, prediccion))

              precision    recall  f1-score   support

         0.0       0.81      0.90      0.85      2880
         1.0       0.53      0.34      0.42       952

    accuracy                           0.76      3832
   macro avg       0.67      0.62      0.63      3832
weighted avg       0.74      0.76      0.74      3832

[[2589  291]
 [ 626  326]]


In [280]:
forest_clf = pickle.load(open('RandomForest.pkl', 'rb'))
prediccion = forest_clf.predict(X_test_df)
print(classification_report(y_test, prediccion))
print(confusion_matrix(y_test, prediccion))

              precision    recall  f1-score   support

         0.0       0.81      0.91      0.86      2880
         1.0       0.57      0.36      0.44       952

    accuracy                           0.77      3832
   macro avg       0.69      0.63      0.65      3832
weighted avg       0.75      0.77      0.75      3832

[[2620  260]
 [ 611  341]]


In [306]:
modelo = keras.models.load_model('NN_Keras')
predictions = modelo.predict(X_test_df)
for i in range(len(predictions)):
    if predictions[i][0] > 0.5:
        predictions[i][0] = 1
    else:
        predictions[i] = 0
print(classification_report(y_test, predictions))
print(confusion_matrix(y_test, predictions))

              precision    recall  f1-score   support

         0.0       0.81      0.87      0.84      2880
         1.0       0.49      0.39      0.44       952

    accuracy                           0.75      3832
   macro avg       0.65      0.63      0.64      3832
weighted avg       0.73      0.75      0.74      3832

[[2498  382]
 [ 581  371]]


A partir de las matrices de confusión podemos determinar cómo se comporta cada modelo y cuál puede ser mejor. 

1. De las matrices vemos que la red neuronal es la que mayor cantidad de casos es capaz de predecir que tienen intención de cambiar de empleo, y la que menos casos de personas con intención real de cambio que son consideradas como no interesadas. En contrapartida es la que más casos interpreta como personas con intención de cambiar cuando realmente no tienen esa intención. Si lo comparamos con los resultados obtenidos del dataset de entrenamiento, los resultados son bastante peores, pudiendo estar la red sobreajustada a los datos de entrenamiento
2. El modelo de KNeighbor es el siguiente modelo en predicción correcta de casos con intención de cambio, pero al igual que la red neuronal tiene bastantes casos de falso rechazo y falso positivo.
3. Finalmente, los modelos Random Forest y XGBoost son los que menos casos reales con intención de cambio detectan, pero los que cometen menos erorres de falso OK, personas sin intención de cambio y predecidas como con cambio. 

Para la selección del modelo se atenderá a la condición de penalizar la detección de más casos con intención real, a cambio de no tener una alta tasa de falso OK. El modelo elegido será el Random Forest, que cargaremos en IBM Cloud.